In [1]:
import fasttext
import re
import ast
from tqdm import tqdm
import gc
import os
import pickle
import random
import nltk
from preprocess_ import preprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from nltk.tokenize  import word_tokenize
import tensorflow as tf
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report,f1_score
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM,Conv1D, GlobalMaxPooling1D,Flatten,MaxPooling1D,GRU,SpatialDropout1D,Bidirectional , BatchNormalization
# pd.set_option('display.max_columns',None)
# pd.set_option('display.max_rows',None)
# pd.set_option('display.max_colwidth',-1)
from sklearn.feature_extraction.text import TfidfVectorizer

tqdm.pandas()

In [2]:
def cleaner(x):
    x = re.sub(r"([0-9]+(\.[0-9]+)?)",r" \1 ", x).strip()
    x = re.sub('([.,!?؟()!\\/\+\-\=\|:"،])', r' \1 ', x)

    x = x.replace('\u200c', ' ').replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')
    
    x = re.sub('\/', ' ', x)
    x = re.sub('«', ' ', x)
    x = re.sub('»', ' ', x)
    
    x = re.sub('\s{2,}', ' ', x)
    return x.strip()

In [3]:
files = os.listdir(r"E:\Work\Sigmoyd\author identification\zoomit\batch data")
li = []


for i in files:
    df = pd.read_csv(f"E:\\Work\\Sigmoyd\\author identification\\zoomit\\batch data\\{i}", index_col=0)
    li.append(df)
    
    
df = pd.concat(li)
df.head()

,category,lead,text,author,title
0,نمایشگاه خودرو ژنو,سوبارو جدیدترین عضو سری مفهومی ویزیو، استیشن و...,شاید بتوان گفت که مدل مفهومی ویزیو تورر (Viziv...,نویسنده: میلاد زنگانه,استیشن مفهومی سوبارو ویزیو تورر معرفی شد
1,نمایشگاه خودرو ژنو,مدل مفهومی کراس‌اور تمام برقی پورشه میشن E با ...,مقامات پورشه علاوه بر رونمایی فیس‌لیفت جدیدی ا...,نویسنده: میلاد زنگانه,کراس اور مفهومی پورشه کراس توریسمو رونمایی شد
2,نمایشگاه خودرو ژنو,نسل پنجم تویوتا سوپرا با مدل مفهومی و مسابقه‌ا...,سرانجام پس از فاش شدن تصاویر اولین مدل مفهومی ...,نویسنده: میلاد زنگانه,کانسپت مسابقه‌ای از نسل جدید تویوتا سوپرا GR ر...
3,موبایل,هواوی پس از عرضه مدل‌های میت ۱۰ در سال جاری، ح...,مدت کوتاهی پس از رونمایی رسمی میت ۱۰ پرو و میت...,نویسنده: مهسا اباذری,هواوی میت SE، نسخه ارتقاءیافته آنر 7 ایکس با ق...
4,پردازنده گرافیکی,شرکت آرم دیروز از طراحی مرجع ۴ تراشه‌ی جدید، ش...,آرم که در سال ۲۰۱۶ به تصاحب هولدینگ چندملیتی س...,نویسنده: میلاد میرکانی,آرم از طراحی مرجع واحدهای پردازش گرافیکی Mali-...


In [4]:
best_authors = df.author.value_counts().head(23).index
df = df[df.author.isin(best_authors)]
df = df.reset_index(drop=True)
df

,category,lead,text,author,title
0,نمایشگاه خودرو ژنو,مدل کوپه از سری GT، اولین خودروی اختصاصی با تو...,بعد از چند ماه انتظار و تماشای تصاویر جاسوسی ا...,نویسنده: محمدرضا اشتری,مرسدس بنز AMG GT نسخه‌ی سدان معرفی شد
1,نمایشگاه خودرو ژنو,خودروی مفهومی بی‌ام‌و M8 گرن‌ کوپه در نمایشگاه...,خودروی مفهومی بی‌ام‌و M8 گرن‌ کوپه معنای حروف ...,نویسنده: ابوالقاسم اسلامی,خودروی مفهومی بی‌ ام‌ و M8 گرن‌کوپه معرفی شد
2,نمایشگاه خودرو ژنو,برابوس خود را برای نمایش دو مدل سدان و کوپه از...,برابوس (Brabus) یکی از بزرگ‌ترین شرکت‌های تیون...,نویسنده: ابوالقاسم اسلامی,سدان و کوپه مرسدس AMG S63 با تیونینگ برابوس مع...
3,نمایشگاه خودرو ژنو,در آستانه نمایشگاه خودروی ژنو ۲۰۱۸، استون مارت...,برند لاگوندا (Lagonda) هنوز زنده است و استون م...,نویسنده: ابوالقاسم اسلامی,خودروی مفهومی الکتریکی استون مارتین لاگوندا وی...
4,نمایشگاه خودرو ژنو,نسخه هیبریدی خودروی بنتلی بنتایگا مدل ۲۰۱۹ با ...,به نظر می‌رسد که بنتلی، ترکیب رانندگی مدرن و ت...,نویسنده: ابوالقاسم اسلامی,بنتلی بنتایگا پلاگین هیبریدی مدل 2019 معرفی شد
5,کسب و کار,چند سالی است که میزهای ایستاده جای خود را در ...,اگر جستجویی ساده در فروشگاه‌های ملزومات اداری ...,نویسنده: مهدی زارع سریزدی,آسیب‌های جسمی ایستادن به مدت طولانی در محیط کار
6,اخبار خودرو,فولکس‌واگن در نمایشگاه ژنو ۲۰۱۸ از اتوبوس مفهو...,یک سال پس از نمایش جهانی اتوبوس سدریک (SEDRIC)...,نویسنده: ابوالقاسم اسلامی,مدل مفهومی سدریک فولکس‌واگن رونمایی شد
7,نمایشگاه خودرو ژنو,اشکودا در نمایشگاه ژنو ۲۰۱۸ از خودروی مفهومی و...,سبک طراحی اشکودا (Skoda) با ساخت هر مدل مفهومی...,نویسنده: ابوالقاسم اسلامی,خودروی مفهومی ویژن ایکس اشکودا رونمایی شد
8,اینترنت و شبکه,هواوی هفته‌ی گذشته و در جریان برگزاری کنگره‌ی ...,هواوی ۲۵ فوریه (۶ اسفندماه) طی کنفرانس جهانی م...,نویسنده: حسین خلیلی صفا,هو‌اوی در MWC 2018 از جدیدترین دستاوردهای 5G خ...
9,نمایشگاه خودرو ژنو,خودروی مرسدس AMG C43 مدل ۲۰۱۹ با تغییراتی در پ...,بعد از این‌که مدل‌های کلاس C مرسدس بنز برای سا...,نویسنده: ابوالقاسم اسلامی,سدان مرسدس بنز AMG C43 مدل 2019 رونمایی شد


In [5]:
df["text"] = df.title + " . " + df.lead + " . " + df.text

In [6]:
prep = preprocess()
df['text'] = df['text'].astype(str).progress_apply(lambda x: cleaner(x))

100%|██████████████████████████████████████████████████████████████████████████| 43458/43458 [00:05<00:00, 7364.75it/s]


In [7]:
df.author = df.author.map(dict(zip(df.author.unique() , range(23))))

In [8]:
vectorizer = TfidfVectorizer(max_features=30000)
X = vectorizer.fit_transform(df['text'].values)
Y = df['author'].values

In [9]:
X.shape

(43458, 30000)

In [14]:
def create_model():
    model = Sequential()
    model.add(Dense(512,activation='relu', input_dim = 30000))
    model.add(Dropout(0.3))
    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(23,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [15]:
model=create_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               15360512  
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                32832     
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 23)                1495      
Total params: 15,394,839
Trainable params: 15,394,839
Non-trainable params: 0
_________________________________________________________________


In [12]:
enc = OneHotEncoder()

Y = enc.fit_transform(Y.reshape(-1,1)).toarray()
Y

D:\anaconda\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [17]:
ACC,AUC =[],[]
pred=[]
valid=[]
Y_=[]

predprob_li=[]
batch_size = 32
epoches = 15
fold = 0
i = 0

# kf=KFold(n_splits=5,random_state=2)
kf = StratifiedKFold(n_splits=5,random_state=2)
y = df['author'].values

X = X.toarray()
for train_index,valid_index in kf.split(X , y):
    Xtrain,Xvalid = X[train_index],X[valid_index]
    Ytrain,Yvalid = Y[train_index],Y[valid_index]
    valid.append(valid_index)
    model=create_model()
    
    history= model.fit(Xtrain, Ytrain, validation_data=(Xvalid,Yvalid),epochs=epoches, batch_size=batch_size, verbose=1)
    #class_weight = class_weights
#     model.save_weights('KFoldModel_tuned\model_'+str(i)+'.h5')
    
    pred_ = model.predict(Xvalid)
    i+=1
#     for i in pred_:
#         pred.append(np.argmax(i))
#         all_preds.append(i)
        
        
#     for i in Yvalid:
#         Y_.append(np.argmax(i))
#         all_Y.append(i)

    gc.collect()
    
    fold+=1
    print("\n\nacc fold ", fold)

Epoch 1/15
1087/1087 [==============================] - 6s 4ms/step - loss: 1.0025 - accuracy: 0.7138 - val_loss: 0.6619 - val_accuracy: 0.8230
Epoch 2/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.1921 - accuracy: 0.9478 - val_loss: 0.6908 - val_accuracy: 0.8203
Epoch 3/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.0686 - accuracy: 0.9820 - val_loss: 0.6994 - val_accuracy: 0.8362
Epoch 4/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.0340 - accuracy: 0.9915 - val_loss: 0.7968 - val_accuracy: 0.8258
Epoch 5/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.0222 - accuracy: 0.9946 - val_loss: 0.8624 - val_accuracy: 0.8311
Epoch 6/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.0163 - accuracy: 0.9960 - val_loss: 1.0888 - val_accuracy: 0.8063
Epoch 7/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.0124 - accuracy: 0.9969 - val_loss: 1.0757 - val_accuracy:

1087/1087 [==============================] - 4s 4ms/step - loss: 0.0063 - accuracy: 0.9988 - val_loss: 0.5294 - val_accuracy: 0.9006
Epoch 13/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.0050 - accuracy: 0.9985 - val_loss: 0.6114 - val_accuracy: 0.8926
Epoch 14/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.0063 - accuracy: 0.9981 - val_loss: 0.6469 - val_accuracy: 0.8925
Epoch 15/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.0037 - accuracy: 0.9993 - val_loss: 0.6399 - val_accuracy: 0.8988


acc fold  4
Epoch 1/15
1087/1087 [==============================] - 5s 5ms/step - loss: 0.9563 - accuracy: 0.7321 - val_loss: 0.6182 - val_accuracy: 0.8230
Epoch 2/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.1593 - accuracy: 0.9564 - val_loss: 0.6354 - val_accuracy: 0.8278
Epoch 3/15
1087/1087 [==============================] - 4s 4ms/step - loss: 0.0571 - accuracy: 0.9860 - val_loss: 0.6992 - val_acc

KeyboardInterrupt: 

In [18]:
(8362 + 8935 + 9068 + 9014 + 8348)/5

8745.4

In [23]:
for i in list(X[2]):
    if i>0:
        print(i)

0.27538075565023384
0.03314953635233994
0.22557472737923867
0.05075606052415437
0.029331001311791328
0.03614693332986591
0.03578783074679995
0.012143476817175182
0.03214481081881698
0.026669464892744726
0.025389054902894715
0.01390607383536901
0.012030911608746933
0.010178637305430704
0.03623262263882511
0.07930899924267323
0.021970449696018107
0.02146284426827676
0.03838973153855235
0.030315164481869772
0.030301130448572057
0.011259159364301477
0.03224519054692223
0.011990820500988736
0.02321751191677827
0.015646346653759635
0.0366833053691258
0.018635755254029298
0.014225903262061947
0.007492243738086992
0.031167471361312538
0.016026533444624645
0.018538613768111837
0.04836703347321787
0.0333207274368388
0.022063567991001704
0.023323734994530054
0.025959631625064778
0.5718539665961181
0.023309228652728185
0.012358926697952802
0.018419652138840663
0.01205986167669929
0.009969242641366081
0.10593137389351766
0.07398086561719594
0.007833616920276964
0.02823970379558746
0.013092485608259